<a href="https://colab.research.google.com/github/akankshakusf/Project-DeepLearning-Sentiment-Analysis-of-IMDB-Movie-Reviews/blob/master/Sentiment_Analysis_with_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# import ML packages
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pathlib
import io
import string
import time
import re
import numpy as random
import gensim.downloader as api
from PIL import Image
from sklearn.metrics import confusion_matrix, roc_curve

# import tf DL packages
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv1D, InputLayer,BatchNormalization, Bidirectional, Dense, Flatten,Dropout,Input,Embedding,TextVectorization
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy, TopKCategoricalAccuracy, CategoricalAccuracy,SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive, files
from tensorboard.plugins import projector

In [26]:
# !pip install --upgrade numpy
# !pip install --upgrade gensim

# Data Preparation

In [27]:
train_ds,val_ds,test_ds=tfds.load('imdb_reviews', split=['train', 'test[:50%]', 'test[50%:]'],as_supervised=True)

In [7]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [28]:
for review,label in val_ds.take(2):
  print(review)
  print(label)

tf.Tensor(b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.", shape=(), dtype=string)
tf.Tensor(1, shape=(),

First we standarize all the values
reference : https://www.tensorflow.org/api_docs/python/tf/strings/regex_replace
- https://github.com/google/re2
- convert to input lowercase
- review html tags through regex (regular expression)
- take off punctuations
- remove special chracters
- remove accented chracters
- reduce the word to root through one of these:
  - Stemming
    - stemming : PorterStemmer
    - lemmatization : Lematize
  - Tokenization
    - chracter tokenization
    - word tokenization
    - subword tokenization
    - n-gram tokenization
  - Vectorization
    - one-hot
    - bag of words
    - tf-idf
    - embeddings
    

- Standardization

In [29]:
def standardization(input_data):
  '''
  Input: raw reviews
  output: standardized reviews
  '''
  lowercase = tf.strings.lower(input_data)
  no_tag = tf.strings.regex_replace(lowercase,"<[^>]+>","")
  output=tf.strings.regex_replace(no_tag,"[%s]"%re.escape(string.punctuation),"")

  return output


In [30]:
standardization("There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.")

<tf.Tensor: shape=(), dtype=string, numpy=b'there are films that make careers for george romero it was night of the living dead for kevin smith clerks for robert rodriguez el mariachi add to that list onur tukels absolutely amazing dingalingless flawless filmmaking and as assured and as professional as any of the aforementioned movies i havent laughed this hard since i saw the full monty and even then i dont think i laughed quite this hard so to speak tukels talent is considerable dingalingless is so chock full of double entendres that one would have to sit down with a copy of this script and do a linebyline examination of it to fully appreciate the uh breadth and width of it every shot is beautifully composed a clear sign of a surehanded director and the performances all around are solid theres none of the overthetop scenery chewing one mightve expected from a film like this dingalingless is a film whose time has come'>

- Stemming

In [31]:
from nltk.stem.porter import PorterStemmer

In [23]:
PorterStemmer().stem("discussion")

'discuss'